In [5]:
# TUTAJ PRÓBY

import pandas as pd
from pyspark.sql import SparkSession
from pyspark.sql.functions import from_json, col, lit
from pyspark.sql.types import StructType, StructField, StringType, DoubleType, TimestampType, BooleanType
import os

output_path = os.path.join(os.getcwd(), "..", 'data', "output")
checkpoint_location = os.path.join(os.getcwd(), "..", 'data', "checkpoint")

# Initialize Spark session with required Kafka packages
scala_version = '2.12'
spark_version = '3.5.1'
packages = [
    f'org.apache.spark:spark-sql-kafka-0-10_{scala_version}:{spark_version}',
    f'org.apache.spark:spark-streaming-kafka-0-10_{scala_version}:{spark_version}',
    'org.apache.kafka:kafka-clients:3.7.0',
    'org.apache.commons:commons-pool2:2.12.0'
]

spark = SparkSession.builder \
    .master("local") \
    .appName("kafka-example") \
    .config("spark.jars.packages", ",".join(packages)) \
    .config("spark.hadoop.fs.file.impl", "org.apache.hadoop.fs.LocalFileSystem") \
    .getOrCreate()

spark.sparkContext.setLogLevel('ERROR')

# Define the schema for the incoming data
schema = StructType([
    StructField("flight_number", StringType(), True),
    StructField("time", TimestampType(), True),
    StructField("longitude", DoubleType(), True),
    StructField("latitude", DoubleType(), True),
    StructField("altitude", DoubleType(), True),
    StructField("event", StringType(), True)
])

# Read from the Kafka topic
kafka_df = spark.readStream \
    .format("kafka") \
    .option("kafka.bootstrap.servers", "localhost:9092") \
    .option("subscribe", "my_topic") \
    .option("startingOffsets", "earliest") \
    .load()

# Extract the value field from the Kafka message and parse it as JSON
parsed_df = kafka_df.selectExpr("CAST(value AS STRING) as json_value") \
    .select(from_json(col("json_value"), schema).alias("data")) \
    .select("data.*")

# Combine with the DataFrame
csv_path = os.path.join(os.getcwd(), "..", 'data', 'flight_data.csv')
flights_df = pd.read_csv(csv_path)
flights_df = flights_df[["FlightNo"]]
flights_df.columns = ['flight_number']  
flights_df[["time", "longtitude", "latitude", "altitude", "event"]] = None
flights_df["has_landed"] = False

# Define schema for the DataFrame
schema = StructType([
    StructField("flight_number", StringType(), True),
    StructField("time", TimestampType(), True),
    StructField("longitude", DoubleType(), True),
    StructField("latitude", DoubleType(), True),
    StructField("altitude", DoubleType(), True),
    StructField("event", StringType(), True),
    StructField("has_landed", BooleanType(), True)
])

# Create a Spark DataFrame with the required schema
spark_flights_df = spark.createDataFrame(flights_df, schema=schema)

# Initialize other columns to null
spark_flights_df = spark_flights_df.withColumn("time", lit(None).cast("timestamp")) \
    .withColumn("longitude", lit(None).cast("double")) \
    .withColumn("latitude", lit(None).cast("double")) \
    .withColumn("altitude", lit(None).cast("double")) \
    .withColumn("event", lit(None).cast("string"))


query_result = result.writeStream \
    .outputMode("append") \
    .format("csv") \
    .option("path", output_path) \
    .option("checkpointLocation", checkpoint_location) \
    .start()


query = parsed_df.writeStream \
    .outputMode("append") \
    .format("console") \
    .start()


# Define query to update chosen flights in spark_flights_df
# query = parsed_df.writeStream \
#     .foreachBatch(lambda batch_df, batch_id: 
#                   batch_df.join(spark_flights_df, 
#                                 batch_df.spark_flight_number == spark_flights_df.flight_number, 
#                                 "inner")
#                           .write.mode("overwrite").csv(output_path)) \
#     .start()

# query = parsed_df.writeStream \
#     .foreachBatch(lambda batch_df, batch_id: 
#                   batch_df.foreach(lambda row: 
#                                     spark_flights_df \
#                                         .filter(spark_flights_df.flight_number == row.flight_number) \
#                                         .write \
#                                         .format("csv") \
#                                         .mode("overwrite") \
#                                         .option("path", output_path) \
#                                         .save())) \
#     .start()


# query.awaitTermination()


Traceback (most recent call last):
  File "C:\Users\vxthe\AppData\Local\Programs\Python\Python310\lib\site-packages\pyspark\serializers.py", line 459, in dumps
    return cloudpickle.dumps(obj, pickle_protocol)
  File "C:\Users\vxthe\AppData\Local\Programs\Python\Python310\lib\site-packages\pyspark\cloudpickle\cloudpickle_fast.py", line 73, in dumps
    cp.dump(obj)
  File "C:\Users\vxthe\AppData\Local\Programs\Python\Python310\lib\site-packages\pyspark\cloudpickle\cloudpickle_fast.py", line 632, in dump
    return Pickler.dump(self, obj)
  File "C:\Users\vxthe\AppData\Local\Programs\Python\Python310\lib\site-packages\pyspark\context.py", line 466, in __getnewargs__
    raise PySparkRuntimeError(
pyspark.errors.exceptions.base.PySparkRuntimeError: [CONTEXT_ONLY_VALID_ON_DRIVER] It appears that you are attempting to reference SparkContext from a broadcast variable, action, or transformation. SparkContext can only be used on the driver, not in code that it run on workers. For more inform

In [3]:
query_result.stop()
query.stop()

In [10]:
spark_flights_df.createOrReplaceTempView("flights")
result = spark.sql("SELECT * FROM flights where flight_number is not null")

# Show the result
result.show(truncate=False)

Py4JJavaError: An error occurred while calling o106.showString.
: org.apache.spark.SparkException: Job aborted due to stage failure: Task 0 in stage 1.0 failed 1 times, most recent failure: Lost task 0.0 in stage 1.0 (TID 1) (LAPTOP-MMD4V0V2.home executor driver): org.apache.spark.SparkException: Python worker failed to connect back.
	at org.apache.spark.api.python.PythonWorkerFactory.createSimpleWorker(PythonWorkerFactory.scala:203)
	at org.apache.spark.api.python.PythonWorkerFactory.create(PythonWorkerFactory.scala:109)
	at org.apache.spark.SparkEnv.createPythonWorker(SparkEnv.scala:124)
	at org.apache.spark.api.python.BasePythonRunner.compute(PythonRunner.scala:174)
	at org.apache.spark.api.python.PythonRDD.compute(PythonRDD.scala:67)
	at org.apache.spark.rdd.RDD.computeOrReadCheckpoint(RDD.scala:367)
	at org.apache.spark.rdd.RDD.iterator(RDD.scala:331)
	at org.apache.spark.rdd.MapPartitionsRDD.compute(MapPartitionsRDD.scala:52)
	at org.apache.spark.rdd.RDD.computeOrReadChe